# Volume Compare
Plots your .obj file from surface classifier on the same axis as common objects and a to scale human body. Helpful for gauging relative size.

Run the cell below, and a file dialogue will appear for you to select your .obj file to plot in 3D. Feel free to use the calculated volume proportions, pictures, and qualitative assessments of their relative size in the results and discussion in your report where relevant. 

**NOTE:** Currently this tool does not work on EIT, it can only be ran locally! A MATLAB version of this tool is available in the `MATLAB` folder.


In [ ]:
import numpy as np
import plotly.graph_objects as go
import tkinter as tk
from tkinter import filedialog


def meshgrid_to_tris(x, y, z):
    n, m = x.shape
    i_list, j_list, k_list = [], [], []
    for ii in range(n - 1):
        for jj in range(m - 1):
            idx = ii * m + jj
            idx_right = ii * m + (jj + 1)
            idx_down = (ii + 1) * m + jj
            idx_diag = (ii + 1) * m + (jj + 1)
            # First triangle of the grid cell
            i_list.append(idx)
            j_list.append(idx_right)
            k_list.append(idx_down)
            # Second triangle
            i_list.append(idx_right)
            j_list.append(idx_diag)
            k_list.append(idx_down)
    return i_list, j_list, k_list

def read_obj(filename):
    vertices = []
    faces = []
    with open(filename, 'r') as file:
        for line in file:
            if line.startswith('v '):
                parts = line.strip().split()[1:]
                vertices.append([float(x) for x in parts])
            elif line.startswith('f '):
                parts = line.strip().split()[1:]
                # Handles faces like "f 1 2 3" or "f 1/1/1 2/2/2 3/3/3"
                face = [int(part.split('/')[0]) - 1 for part in parts]
                faces.append(face)
    return np.array(vertices), np.array(faces)

def compute_mesh_volume(vertices, faces):
    volume = 0.0
    for face in faces:
        v1 = vertices[face[0]]
        v2 = vertices[face[1]]
        v3 = vertices[face[2]]
        volume += np.dot(v1, np.cross(v2, v3)) / 6.0
    return abs(volume)

def create_sphere_trace(volume, center, color, opacity=0.5, resolution=30):
    r = (3 * volume / (4 * np.pi))**(1/3)
    u = np.linspace(0, 2 * np.pi, resolution)
    v = np.linspace(0, np.pi, resolution)
    u_grid, v_grid = np.meshgrid(u, v)
    x = r * np.cos(u_grid) * np.sin(v_grid) + center[0]
    y = r * np.sin(u_grid) * np.sin(v_grid) + center[1]
    z = r * np.cos(v_grid) + center[2]
    i, j, k = meshgrid_to_tris(x, y, z)
    return go.Mesh3d(
        x=x.flatten(),
        y=y.flatten(),
        z=z.flatten(),
        i=i, j=j, k=k,
        color=color,
        opacity=opacity,
        flatshading=True,
        showscale=False
    )

def create_ellipsoid_trace(volume, center, color, opacity=0.5, resolution=30):
    scale = (volume / ((4/3) * np.pi))**(1/3)
    rx, ry, rz = 1 * scale, 1.5 * scale, 1.2 * scale
    u = np.linspace(0, 2 * np.pi, resolution)
    v = np.linspace(0, np.pi, resolution)
    u_grid, v_grid = np.meshgrid(u, v)
    x = rx * np.cos(u_grid) * np.sin(v_grid) + center[0]
    y = ry * np.sin(u_grid) * np.sin(v_grid) + center[1]
    z = rz * np.cos(v_grid) + center[2]
    i, j, k = meshgrid_to_tris(x, y, z)
    return go.Mesh3d(
        x=x.flatten(),
        y=y.flatten(),
        z=z.flatten(),
        i=i, j=j, k=k,
        color=color,
        opacity=opacity,
        flatshading=True,
        showscale=False
    )

def create_imported_object_trace(obj, reference_center, reference_radius, color='red', opacity=0.5):

    vertices = obj['vertices']
    faces = obj['faces']
    min_coords = np.min(vertices, axis=0)
    max_coords = np.max(vertices, axis=0)
    obj_center = (min_coords + max_coords) / 2
    obj_size = max_coords - min_coords
    obj_radius = np.max(obj_size) / 2
    offset = reference_radius + obj_radius
    new_position = np.array(reference_center) + np.array([offset - obj_center[0],
                                                            -obj_center[1],
                                                            -obj_center[2]])
    translated_vertices = vertices + new_position
    return go.Mesh3d(
        x=translated_vertices[:, 0],
        y=translated_vertices[:, 1],
        z=translated_vertices[:, 2],
        i=faces[:, 0],
        j=faces[:, 1],
        k=faces[:, 2],
        color=color,
        opacity=opacity,
        flatshading=True,
        showscale=False
    )

def create_imported_object_in_abdomen_trace(obj, color='red', opacity=0.5):
    vertices = obj['vertices']
    # no rotate
    obj_center = np.mean(vertices, axis=0)
    abdomen_center = np.array([0, 80, 0])
    abdomen_radius= 15
    offset = abdomen_radius
    new_position = abdomen_center - obj_center + np.array([0, -offset, 0])
    translated_vertices = vertices + new_position

    return go.Mesh3d(
        x=translated_vertices[:, 0],
        y=translated_vertices[:, 1],
        z=translated_vertices[:, 2],
        i=obj['faces'][:, 0],
        j=obj['faces'][:, 1],
        k=obj['faces'][:, 2],
        color=color,
        opacity=opacity,
        flatshading=True,
        showscale=False
    )

def create_human_outline_trace():

    human_height = 175
    head_radius = 0.15 * human_height        
    head_center = np.array([0, human_height - head_radius])  
    shoulder_y = human_height - 2 * head_radius  
    torso_length = 0.5 * human_height        
    waist_y = shoulder_y - torso_length       
    shoulder_width = 0.5 * human_height      
    waist_width = 0.3 * human_height          
    leg_start_y = waist_y - 0.05 * human_height 
    leg_length = 0.45 * human_height          
    arm_length = 0.4 * human_height          

    z = 0  

    x_segments = []
    y_segments = []
    z_segments = []

    theta = np.linspace(0, 2*np.pi, 100)
    x_head = head_center[0] + head_radius * np.cos(theta)
    y_head = head_center[1] + head_radius * np.sin(theta)
    z_head = np.full_like(theta, z)
    x_segments.extend(x_head.tolist())
    y_segments.extend(y_head.tolist())
    z_segments.extend(z_head.tolist())
    x_segments.append(None)
    y_segments.append(None)
    z_segments.append(None)

    left_shoulder = (-shoulder_width/2, shoulder_y)
    left_waist = (-waist_width/2, waist_y)
    x_segments.extend([left_shoulder[0], left_waist[0]])
    y_segments.extend([left_shoulder[1], left_waist[1]])
    z_segments.extend([z, z])
    x_segments.append(None)
    y_segments.append(None)
    z_segments.append(None)
    right_shoulder = (shoulder_width/2, shoulder_y)
    right_waist = (waist_width/2, waist_y)
    x_segments.extend([right_shoulder[0], right_waist[0]])
    y_segments.extend([right_shoulder[1], right_waist[1]])
    z_segments.extend([z, z])
    x_segments.append(None)
    y_segments.append(None)
    z_segments.append(None)

    left_arm_end = (left_shoulder[0] - 15, shoulder_y - arm_length)
    x_segments.extend([left_shoulder[0], left_arm_end[0]])
    y_segments.extend([left_shoulder[1], left_arm_end[1]])
    z_segments.extend([z, z])
    x_segments.append(None)
    y_segments.append(None)
    z_segments.append(None)

    right_arm_end= (right_shoulder[0] + 15, shoulder_y - arm_length)
    x_segments.extend([right_shoulder[0], right_arm_end[0]])
    y_segments.extend([right_shoulder[1], right_arm_end[1]])
    z_segments.extend([z, z])
    x_segments.append(None)
    y_segments.append(None)
    z_segments.append(None)

    left_leg_end = (left_waist[0], leg_start_y - leg_length)
    x_segments.extend([left_waist[0], left_leg_end[0]])
    y_segments.extend([left_waist[1], left_leg_end[1]])
    z_segments.extend([z, z])
    x_segments.append(None)
    y_segments.append(None)
    z_segments.append(None)

    right_leg_end = (right_waist[0], leg_start_y - leg_length)
    x_segments.extend([right_waist[0], right_leg_end[0]])
    y_segments.extend([right_waist[1], right_leg_end[1]])
    z_segments.extend([z, z])
    x_segments.append(None)
    y_segments.append(None)
    z_segments.append(None)

    return go.Scatter3d(
        x=x_segments,
        y=y_segments,
        z=z_segments,
        mode='lines',
        line=dict(color='black', width=6),
        showlegend=False
    )


root = tk.Tk()
root.withdraw()
print("Please select the OBJ file (from surfaceclassifier) in the pop up file window.")
filepath = filedialog.askopenfilename(filetypes=[("OBJ files", "*.obj")],
                                      title="Select an OBJ file")
if not filepath:
    print("User canceled file selection.")
else:
    vertices, faces = read_obj(filepath)
    vertices = vertices * 0.1  # scale fact
    imported_volume = compute_mesh_volume(vertices, faces)

    min_coords = np.min(vertices, axis=0)
    max_coords = np.max(vertices, axis=0)
    obj_size = max_coords - min_coords
    obj_radius = np.max(obj_size) / 2

    volumes = {
        'tennis_ball': 4/3 * np.pi * (6.7/2)**3,      # Tennis ball (6.7 cm diameter)
        'football':   4/3 * np.pi * (11/2)**2 * (28/2),  # Football (11 cm minor axis, 28 cm major axis)
        'basketball': 4/3 * np.pi * (24.6/2)**3,         # Basketball (24.6 cm diameter)
        'abdomen':    4/3 * np.pi * (20/2) * (35/2) * (25/2)  # Human abdomen (approximate ellipsoid)
    }

    print("Compared to:")
    print(f"  - Tennis Ball: {imported_volume / volumes['tennis_ball']:.2f}x")
    print(f"  - Football: {imported_volume / volumes['football']:.2f}x")
    print(f"  - Basketball: {imported_volume / volumes['basketball']:.2f}x")
    print(f"  - Human Abdomen: {imported_volume / volumes['abdomen']:.2f}x")

    obj = {'vertices': vertices, 'faces': faces}

    common_layout = dict(
        scene=dict(
            aspectmode="data",  
            xaxis=dict(visible=False, showgrid=False, zeroline=False),
            yaxis=dict(visible=False, showgrid=False, zeroline=False),
            zaxis=dict(visible=False, showgrid=False, zeroline=False)
        ),
        margin=dict(l=0, r=0, t=40, b=0)
    )

    print('Figure 1: Imported Object vs. Tennis Ball'
    tennis_radius = (3 * volumes['tennis_ball'] / (4 * np.pi))**(1/3)
    fig1 = go.Figure()
    fig1.add_trace(create_sphere_trace(volumes['tennis_ball'], [0, 0, 0], 'green'))
    fig1.add_trace(create_imported_object_trace(obj, [0, 0, 0], tennis_radius, color='red'))
    fig1.update_layout(title="Imported Object vs. Tennis Ball", **common_layout)
    fig1.show()

    print('Figure 2: Imported Object vs. Basketball')
    basketball_radius = (3 * volumes['basketball'] / (4 * np.pi))**(1/3)
    fig2 = go.Figure()
    fig2.add_trace(create_sphere_trace(volumes['basketball'], [0, 0, 0], 'blue'))
    fig2.add_trace(create_imported_object_trace(obj, [0, 0, 0], basketball_radius, color='red'))
    fig2.update_layout(title="Imported Object vs. Basketball", **common_layout)
    fig2.show()

    print('Figure 3: Imported Object vs. Football')
    football_radius = (volumes['football'] / ((4/3) * np.pi))**(1/3)
    fig3 = go.Figure()
    fig3.add_trace(create_ellipsoid_trace(volumes['football'], [0, 0, 0], 'magenta'))
    fig3.add_trace(create_imported_object_trace(obj, [0, 0, 0], football_radius, color='red'))
    fig3.update_layout(title="Imported Object vs. Football", **common_layout)
    fig3.show()

    print('Figure 4: Imported Object in Human Abdomen with Human Outline')
    fig4 = go.Figure()
    fig4.add_trace(create_human_outline_trace())
    fig4.add_trace(create_imported_object_in_abdomen_trace(obj, color='red'))
    fig4.update_layout(title="Imported Object in Human Abdomen", **common_layout)
    fig4.show()
    Print()
